In [1]:
from pyspark.sql import SparkSession
from pyspark.ml.recommendation import ALS
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.sql import functions as F
spark = SparkSession.builder.appName('everything').getOrCreate()

23/11/02 07:13:26 WARN Utils: Your hostname, Ubuntu resolves to a loopback address: 127.0.1.1; using 10.0.2.15 instead (on interface enp0s3)
23/11/02 07:13:26 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/11/02 07:13:41 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
df = spark.read.csv('./everything.csv', inferSchema=True,header=True)

print(f'The shape of dataframe : {df.count()} , { len(df.columns)}')

The shape of dataframe : 27306439 , 33


In [3]:
# df = df.dropDuplicates()

# print(f'The shape of dataframe : {df.count()} , { len(df.columns)}')


In [7]:
# df.show(500)

In [8]:
df.printSchema()

root
 |-- article_id: integer (nullable = true)
 |-- customer_id: string (nullable = true)
 |-- FN: double (nullable = true)
 |-- Active: double (nullable = true)
 |-- club_member_status: string (nullable = true)
 |-- fashion_news_frequency: string (nullable = true)
 |-- age: integer (nullable = true)
 |-- postal_code: string (nullable = true)
 |-- count: integer (nullable = true)
 |-- product_code: integer (nullable = true)
 |-- prod_name: string (nullable = true)
 |-- product_type_no: integer (nullable = true)
 |-- product_type_name: string (nullable = true)
 |-- product_group_name: string (nullable = true)
 |-- graphical_appearance_no: integer (nullable = true)
 |-- graphical_appearance_name: string (nullable = true)
 |-- colour_group_code: integer (nullable = true)
 |-- colour_group_name: string (nullable = true)
 |-- perceived_colour_value_id: integer (nullable = true)
 |-- perceived_colour_value_name: string (nullable = true)
 |-- perceived_colour_master_id: integer (nullable = t

In [9]:
df.columns

['article_id',
 'customer_id',
 'FN',
 'Active',
 'club_member_status',
 'fashion_news_frequency',
 'age',
 'postal_code',
 'count',
 'product_code',
 'prod_name',
 'product_type_no',
 'product_type_name',
 'product_group_name',
 'graphical_appearance_no',
 'graphical_appearance_name',
 'colour_group_code',
 'colour_group_name',
 'perceived_colour_value_id',
 'perceived_colour_value_name',
 'perceived_colour_master_id',
 'perceived_colour_master_name',
 'department_no',
 'department_name',
 'index_code',
 'index_name',
 'index_group_no',
 'index_group_name',
 'section_no',
 'section_name',
 'garment_group_no',
 'garment_group_name',
 'detail_desc']

In [10]:
df.show(5)

+----------+--------------------+----+------+------------------+----------------------+---+--------------------+-----+------------+---------+---------------+-----------------+------------------+-----------------------+-------------------------+-----------------+-----------------+-------------------------+---------------------------+--------------------------+----------------------------+-------------+--------------------+----------+------------------+--------------+----------------+----------+--------------------+----------------+------------------+--------------------+
|article_id|         customer_id|  FN|Active|club_member_status|fashion_news_frequency|age|         postal_code|count|product_code|prod_name|product_type_no|product_type_name|product_group_name|graphical_appearance_no|graphical_appearance_name|colour_group_code|colour_group_name|perceived_colour_value_id|perceived_colour_value_name|perceived_colour_master_id|perceived_colour_master_name|department_no|     department_nam

In [14]:
import pyspark.sql.functions as sql_func
from pyspark.sql.types import *
from pyspark.ml.recommendation import ALS, ALSModel
from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession
from pyspark.mllib.evaluation import RegressionMetrics, RankingMetrics
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.feature import StringIndexer

In [16]:
indexer = StringIndexer(inputCol="customer_id", outputCol="user_index")
indexed_user_df = indexer.fit(df).transform(df)

indexer = StringIndexer(inputCol="article_id", outputCol="item_index")
indexed_item_df = indexer.fit(indexed_user_df).transform(indexed_user_df)

23/11/02 07:48:20 ERROR Executor: Exception in task 0.0 in stage 11.0 (TID 300)]
org.apache.spark.SparkException: Kryo serialization failed: Buffer overflow. Available: 0, required: 63
Serialization trace:
_data (org.apache.spark.util.collection.OpenHashSet)
org$apache$spark$util$collection$OpenHashMap$$_keySet (org.apache.spark.util.collection.OpenHashMap$mcJ$sp). To avoid this, increase spark.kryoserializer.buffer.max value.
	at org.apache.spark.serializer.KryoSerializerInstance.serialize(KryoSerializer.scala:446)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$SpecificUnsafeProjection.apply(Unknown Source)
	at org.apache.spark.sql.execution.aggregate.ComplexTypedAggregateExpression.eval(TypedAggregateExpression.scala:260)
	at org.apache.spark.sql.catalyst.expressions.aggregate.TypedImperativeAggregate.eval(interfaces.scala:594)
	at org.apache.spark.sql.execution.aggregate.AggregationIterator.$anonfun$generateResultProjection$5(AggregationIterator.scala:257)
	at org.apac

Py4JJavaError: An error occurred while calling o104.fit.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 11.0 failed 1 times, most recent failure: Lost task 0.0 in stage 11.0 (TID 300) (10.0.2.15 executor driver): org.apache.spark.SparkException: Kryo serialization failed: Buffer overflow. Available: 0, required: 63
Serialization trace:
_data (org.apache.spark.util.collection.OpenHashSet)
org$apache$spark$util$collection$OpenHashMap$$_keySet (org.apache.spark.util.collection.OpenHashMap$mcJ$sp). To avoid this, increase spark.kryoserializer.buffer.max value.
	at org.apache.spark.serializer.KryoSerializerInstance.serialize(KryoSerializer.scala:446)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$SpecificUnsafeProjection.apply(Unknown Source)
	at org.apache.spark.sql.execution.aggregate.ComplexTypedAggregateExpression.eval(TypedAggregateExpression.scala:260)
	at org.apache.spark.sql.catalyst.expressions.aggregate.TypedImperativeAggregate.e

In [12]:
(training, test) = df.randomSplit([0.99, 0.01])

In [13]:
als = ALS(maxIter=2, regParam=0.01, 
          userCol="customer_id", itemCol="article_id", ratingCol="count",
          coldStartStrategy="drop",
          implicitPrefs=True)
model = als.fit(training)

# Evaluate the model by computing the RMSE on the test data
predictions = model.transform(test)
evaluator = RegressionEvaluator(metricName="rmse", labelCol="watching_percentage",
                                predictionCol="prediction")

rmse = evaluator.evaluate(predictions)
print("Root-mean-square error = " + str(rmse))

23/11/02 07:40:29 ERROR Instrumentation: java.lang.IllegalArgumentException: requirement failed: Column customer_id must be of type numeric but was actually of type string.
	at scala.Predef$.require(Predef.scala:281)
	at org.apache.spark.ml.util.SchemaUtils$.checkNumericType(SchemaUtils.scala:78)
	at org.apache.spark.ml.recommendation.ALSParams.validateAndTransformSchema(ALS.scala:259)
	at org.apache.spark.ml.recommendation.ALSParams.validateAndTransformSchema$(ALS.scala:257)
	at org.apache.spark.ml.recommendation.ALS.validateAndTransformSchema(ALS.scala:616)
	at org.apache.spark.ml.recommendation.ALS.transformSchema(ALS.scala:753)
	at org.apache.spark.ml.recommendation.ALS.$anonfun$fit$1(ALS.scala:715)
	at org.apache.spark.ml.util.Instrumentation$.$anonfun$instrumented$1(Instrumentation.scala:191)
	at scala.util.Try$.apply(Try.scala:213)
	at org.apache.spark.ml.util.Instrumentation$.instrumented(Instrumentation.scala:191)
	at org.apache.spark.ml.recommendation.ALS.fit(ALS.scala:714)
	

IllegalArgumentException: requirement failed: Column customer_id must be of type numeric but was actually of type string.